In [3]:
import os
import numpy as np
import mne  # Ensure you have MNE installed
from aeon.classification.convolution_based import RocketClassifier
from sklearn.metrics import accuracy_score, classification_report

def parse_seizure_times_from_file(summary_file_path):
    """
    Parses seizure times from a summary text file.
    
    Parameters:
    - summary_file_path: Path to the summary file.
    
    Returns:
    - seizure_times: List of tuples containing seizure start and end times.
    """
    seizure_times = []
    with open(summary_file_path, 'r') as file:
        lines = file.readlines()
        start_time = None
        for line in lines:
            if 'Seizure Start Time:' in line:
                start_time = int(line.split(':')[1].strip().split()[0])  # Extract number only
            elif 'Seizure End Time:' in line and start_time is not None:
                end_time = int(line.split(':')[1].strip().split()[0])  # Extract number only
                seizure_times.append((start_time, end_time))
                start_time = None  # Reset start_time after capturing a seizure
    return seizure_times

def extract_segments(edf_file_path, seizure_times, segment_duration=2, max_segments=100):
    """
    Extracts segments from the EDF file and labels them.

    Parameters:
    - edf_file_path: Path to the EDF file.
    - seizure_times: List of tuples containing seizure start and end times.
    - segment_duration: Duration of each segment in seconds.
    - max_segments: Maximum number of segments to extract.

    Returns:
    - X: 3D numpy array of shape (n_cases, n_channels, n_timepoints)
    - y: 1D numpy array of labels (0 for interictal, 1 for preictal)
    """
    raw = mne.io.read_raw_edf(edf_file_path, preload=True)
    data = raw.get_data()  # Shape: (n_channels, n_timepoints)
    sfreq = raw.info['sfreq']  # Sampling frequency
    segment_samples = int(segment_duration * sfreq)  # Convert duration to samples
    n_channels = data.shape[0]
    
    X = []
    y = []

    # Extract preictal segments (30 minutes before seizure)
    for start, _ in seizure_times:
        preictal_start = max(start - 1800, 0)  # 30 minutes before the seizure
        preictal_end = start
        
        for i in range(preictal_start, preictal_end, segment_samples):
            if len(X) >= max_segments:
                break
            if i + segment_samples <= preictal_end:  # Check bounds
                segment = data[:, i:i + segment_samples]
                X.append(segment)
                y.append(1)  # Preictal label

    # Extract interictal segments (4 hours before seizure)
    for start, _ in seizure_times:
        interictal_start = max(start - 14400, 0)  # 4 hours before the seizure
        
        for i in range(interictal_start, start - 1800, segment_samples):
            if len(X) >= max_segments:
                break
            if i + segment_samples <= start - 1800:  # Ensure it's before preictal period
                segment = data[:, i:i + segment_samples]
                X.append(segment)
                y.append(0)  # Interictal label

    X = np.array(X)  # Shape: (n_cases, n_channels, n_timepoints)
    y = np.array(y)  # Shape: (n_cases,)
    
    return X, y

def load_data(folder_path, seizure_times, max_segments_per_file=100):
    """
    Load data from a folder containing EDF files and extract segments.

    Parameters:
    - folder_path: Path to the folder containing EDF files.
    - seizure_times: List of tuples containing seizure start and end times.
    - max_segments_per_file: Maximum segments to extract from each file.

    Returns:
    - X: 3D numpy array of shape (n_cases, n_channels, n_timepoints)
    - y: 1D numpy array of labels (0 for interictal, 1 for preictal)
    """
    X = []
    y = []
    
    for filename in os.listdir(folder_path):
        if filename.endswith('.edf'):
            edf_file_path = os.path.join(folder_path, filename)
            X_segments, y_segments = extract_segments(edf_file_path, seizure_times, max_segments=max_segments_per_file)
            X.append(X_segments)
            y.append(y_segments)

    X = np.concatenate(X, axis=0)
    y = np.concatenate(y, axis=0)
    
    return X, y

# Define paths
summary_file_path = "D:/chb-mit-scalp-eeg-database-1.0.0/chb02/chb02-summary.txt"  # Update with your summary file path
folder_path = "D:/chb-mit-scalp-eeg-database-1.0.0/chb02"  # Update with your data folder path

# Load seizure times from the summary file
seizure_times = parse_seizure_times_from_file(summary_file_path)

# Load data
X, y = load_data(folder_path, seizure_times)

# Initialize the RocketClassifier
classifier = RocketClassifier(num_kernels=500)

# Split the data for training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the classifier
classifier.fit(X_train, y_train)

# Predict on the test set
y_test_pred = classifier.predict(X_test)

# Evaluate on the test set
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))


The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.
Extracting EDF parameters from D:\chb-mit-scalp-eeg-database-1.0.0\chb02\chb02_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


C:\Users\HP\AppData\Local\Temp\ipykernel_19908\495003610.py:44: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file_path, preload=True)


Extracting EDF parameters from D:\chb-mit-scalp-eeg-database-1.0.0\chb02\chb02_02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


C:\Users\HP\AppData\Local\Temp\ipykernel_19908\495003610.py:44: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file_path, preload=True)


Extracting EDF parameters from D:\chb-mit-scalp-eeg-database-1.0.0\chb02\chb02_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


...

Extracting EDF parameters from D:\chb-mit-scalp-eeg-database-1.0.0\chb02\chb02_34.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


C:\Users\HP\AppData\Local\Temp\ipykernel_19908\495003610.py:44: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file_path, preload=True)


Extracting EDF parameters from D:\chb-mit-scalp-eeg-database-1.0.0\chb02\chb02_35.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


C:\Users\HP\AppData\Local\Temp\ipykernel_19908\495003610.py:44: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file_path, preload=True)


Test Accuracy: 0.7875
              precision    recall  f1-score   support

           0       0.70      0.88      0.78        34
           1       0.89      0.72      0.80        46

    accuracy                           0.79        80
   macro avg       0.79      0.80      0.79        80
weighted avg       0.81      0.79      0.79        80

